In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib as ta
import mplfinance as mpf
import seaborn as sns
pd.set_option('display.max_columns', None);pd.set_option('display.max_rows', None)

In [ ]:
spac_file='/home/thakur/stock_information/spac_list.csv'
spac=pd.read_csv(spac_file)
spac_list=spac['Symbol'].to_list()
spac_list[:20]

In [ ]:
def plot_with_mpl(root,tick,day=60,sty='classic',typ='candle'):
    "returns the candle stick of a stock tick for the given root file."

    df=pd.read_csv(root+'/'+tick+'.csv').tail(day)
    df['Date']=df['Date'].apply(lambda x:x.split()[0])

    df['rsi']=ta.RSI(df['Close']);df['sma-5']=ta.SMA(df['Close'],5);df['sma-10']=ta.SMA(df['Close'],10)
    #df.fillna(0,inplace=True)
    apds = [ 
         mpf.make_addplot((df['rsi']),panel=1,color='r',secondary_y=True,type='line',width=0.5),
         #mpf.make_addplot((df['sma-5']),color='g',type='line',width=0.2),
         #mpf.make_addplot((df['sma-10']),color='r',type='line',width=0.2)
       ]
    df.Volume=df.Volume/10**6              #100k volume
    df.Date=pd.to_datetime(df.Date)
    df.set_index('Date',inplace=True)
    fig,axes=mpf.plot(df,type=typ,mav=(5,21),volume=True,tight_layout=False,figratio=(80,36),figscale=1,
                    style=sty,returnfig=True,addplot=apds,scale_width_adjustment=dict(volume=0.6,candle=0.8,lines=0.4))#,update_width_config=dict(candle_linewidth=1.75,candle_width=0.1))
    axes[0].legend([" "," ","5-SMA","21-SMA"],loc='upper center')
    #axes[0].legend(loc='upper left')
    axes[0].set_title(tick)
    mpf.show()

# def make_clickable(url, name):
#     return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,name)

def get_yahoo_link(ticker):
    """
    Get the yahoo link
    """
    #return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,name)
    url="https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch"
    #return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,ticker)
    return url.format(ticker,ticker)


In [ ]:
file='/home/thakur/test/combined/all_nasdaq.csv'

In [ ]:
df=pd.read_csv(file)

In [ ]:
df=df[(df.CATEGORY!='nano') & (df.CATEGORY!='micro') & (df['%CHG']!=np.inf) & (df.COUNTRY=='United States')]
df['YAHOO']=df['TICKER'].apply(lambda x:get_yahoo_link(x))#.apply(lambda x: yahoo_clickable(x, 'yahoo'))
df=df.query('HH & HL & HC & CP>0 & VOL>AVGVOL & CP>SMA50 & CP>SMA200')\
.sort_values('CP')\
.reset_index(drop=True)



df[:20]

In [ ]:
df=df.drop(columns=['LH','LL','SMA21','HH','HL','HC','HV'])
df
#df[:20]

In [ ]:
df=df[~df.TICKER.isin(spac_list)]
df

In [ ]:
tick=df['TICKER'].to_list()
plot=True
if plot:
    for c,i in enumerate(tick):
        
        print(f"\n\n{df[df['TICKER']==i].to_string()}\n")
        dis=df[df['TICKER']==i]['CATEGORY'].values[0]
        #print(f"dis: {dis}\n")
        root='/home/thakur/test/'+dis+'/'
        #print(f"root: {root}\n")
        plot_with_mpl(root,i,sty='default')